In [ ]:
# @title 💻 1.1 Check CUDA and Python Version (tested with CUDA 12.2 + Python 3.10.12)

# This notebook is supposed to be connected to NVIDIA drivers with CUDA.
# If this doesn't load check that GPU is selected as hardware accelerator under Edit -> Notebook settings.
!nvidia-smi
!python -V

Sun Mar 16 10:55:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# @title 1.2 👆 Select Checkpoint
!pip install ipywidgets==7.7.1 --quiet

from IPython.display import display
import ipywidgets as widgets
import shutil

ckpt_dic = {
    "Original (higher quality)": "prs-eth/marigold-v1-0",
    "LCM (faster)": "prs-eth/marigold-lcm-v1-0",
}

ckpt_name = 'LCM (faster)'
ckpt_path = ckpt_dic[ckpt_name]
w = widgets.Dropdown(
    options=['Original (higher quality)', 'LCM (faster)'],
    value=ckpt_name,
    description='Checkpoint:',
)


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        ckpt_name = change['new']
        ckpt_path = ckpt_dic[ckpt_name]
        # print(f"Change to checkpoint: {ckpt_name} ({ckpt_path})")

w.observe(on_change)

display(w)
# print(f"{ckpt_path = }")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.8 MB/s eta 0:00:00


Dropdown(description='Checkpoint:', index=1, options=('Original (higher quality)', 'LCM (faster)'), value='LCM…

## 1.3 🏗️ Installation and Initialization

We provide two ways to run the pipeline. You can choose either ONE to run:

### 🅱 1.3-A Using code from the [full repository](https://github.com/prs-eth/marigold)

In [ ]:
# @title 📦 i. Clone repository

%%shell
cd /content

if [ -d "Marigold" ]; then
    cd Marigold
    git pull
else
    git clone https://github.com/prs-eth/Marigold.git
    cd Marigold
fi


Cloning into 'Marigold'...
remote: Enumerating objects: 633, done.
remote: Counting objects: 100% (291/291), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 633 (delta 232), reused 165 (delta 165), pack-reused 342 (from 1)
Receiving objects: 100% (633/633), 5.70 MiB | 17.83 MiB/s, done.
Resolving deltas: 100% (372/372), done.


In [ ]:
# @title 🔨 ii. Install dependencies (might take minutes)
%%shell

cd /content/Marigold

# pip install -r requirements.txt --upgrade  --quiet
pip install accelerate diffusers matplotlib scipy torch transformers --quiet

# for progress bar
pip install ipywidgets==7.7.1 --quiet

pip install -r requirements+.txt -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 1.6 MB/s eta 0:00:00

In [ ]:
# @title 🔧 iii. Setup directories

import os

# Directories
repo_dir = "/content/Marigold"
input_dir = os.path.join(repo_dir, "input")
output_dir = os.path.join(repo_dir, "output")
output_dir_color = os.path.join(output_dir, "depth_colored")
output_dir_tif = os.path.join(output_dir, "depth_bw")
output_dir_npy = os.path.join(output_dir, "depth_npy")

os.chdir(repo_dir)



In [ ]:
# @title 🏇 iv. Initialize pipeline

# change model cache directory
# !export HF_HOME=/content/Marigold/checkpoint

from marigold import MarigoldPipeline

pipe = MarigoldPipeline.from_pretrained(ckpt_path)

pipe = pipe.to("cuda")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

# Path to the zip file in your Google Drive
zip_file_path = '/content/drive/My Drive/datasets/kitti_colab.zip'

# Destination folder where you want to unzip the files
unzip_path = '/content/kitti_colab/'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

# Now you can load the dataset from the unzipped directory
import os
print(os.listdir(unzip_path))


['data_for_colab', 'colab_eigen_test_files_with_gt.txt']


In [ ]:
%%shell

export BASE_DATA_DIR="/content/kitti_colab" # Set target data directory


In [ ]:
%%shell

mv /content/kitti_colab/colab_eigen_test_files_with_gt.txt /content/Marigold/data_split/kitti/
rm /content/Marigold/data_split/kitti/eigen_test_files_with_gt.txt

In [ ]:
%%shell

mv /content/Marigold/data_split/kitti/colab_eigen_test_files_with_gt.txt /content/Marigold/data_split/kitti/eigen_test_files_with_gt.txt


In [ ]:
%%shell

export BASE_DATA_DIR="/content/kitti_colab"

bash script/eval/21_infer_kitti.sh

+ ckpt=prs-eth/marigold-v1-0
+ subfolder=eval
+ python infer.py --checkpoint prs-eth/marigold-v1-0 --seed 1234 --base_data_dir /content/kitti_colab --denoise_steps 4 --ensemble_size 5 --processing_res 768 --dataset_config config/dataset/data_kitti_eigen_test.yaml --output_dir output/eval/kitti_eigen_test/prediction
2025-03-16 12:57:41.534580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742129861.576024   32268 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742129861.588668   32268 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
arguments: Namespace(checkpoint='prs-eth/marigold-v1-0', dataset_config='config/dataset/data_kitti_eigen_test.yaml', bas

In [ ]:
print("Hello world")

Hello world


In [ ]:
!dir


config		 dir.zip	   eval.py	marigold   requirements++.txt  run.py  train.py
CONTRIBUTING.md  doc		   infer.py	output	   requirements+.txt   script
data_split	 environment.yaml  LICENSE.txt	README.md  requirements.txt    src


In [ ]:
%%shell
export BASE_DATA_DIR="/content/kitti_colab"

bash script/eval/22_eval_kitti.sh

+ subfolder=eval
+ python eval.py --base_data_dir /content/kitti_colab --dataset_config config/dataset/data_kitti_eigen_test.yaml --alignment least_square --prediction_dir output/eval/kitti_eigen_test/prediction --output_dir output/eval/kitti_eigen_test/eval_metric
Evaluating: 100% 652/652 [00:46<00:00, 14.01it/s]


In [ ]:
!zip -r kitti_4_5_768.zip /content/Marigold/output/eval/kitti_eigen_test


  adding: content/Marigold/output/eval/kitti_eigen_test/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/eval_metric/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/eval_metric/eval_metrics-least_square.txt (deflated 66%)
  adding: content/Marigold/output/eval/kitti_eigen_test/eval_metric/per_sample_metrics.csv (deflated 63%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/2011_09_26/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/2011_09_26/2011_09_26_drive_0027_sync/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/2011_09_26/2011_09_26_drive_0027_sync/image_02/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/2011_09_26/2011_09_26_drive_0027_sync/image_02/data/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/2011_09_26/2011_

In [ ]:
!cp kitti_4_5_768.zip /content/drive/MyDrive/


In [ ]:
!zip -r kitti_4_5_768_metrics.zip /content/Marigold/output/eval/kitti_eigen_test


  adding: content/Marigold/output/eval/kitti_eigen_test/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/eval_metric/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/eval_metric/eval_metrics-least_square.txt (deflated 66%)
  adding: content/Marigold/output/eval/kitti_eigen_test/eval_metric/per_sample_metrics.csv (deflated 63%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/2011_09_26/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/2011_09_26/2011_09_26_drive_0027_sync/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/2011_09_26/2011_09_26_drive_0027_sync/image_02/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/2011_09_26/2011_09_26_drive_0027_sync/image_02/data/ (stored 0%)
  adding: content/Marigold/output/eval/kitti_eigen_test/prediction/2011_09_26/2011_

In [ ]:
!cp kitti_4_5_768_metrics.zip /content/drive/MyDrive/